In [1]:
%%configure -f
{"heartbeatTimeoutInSecond": "3600"}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1589357389771_0003,pyspark,shutting_down,Link,Link,


In [2]:
df = sqlContext.read.csv('s3a://sparkdemonstration/10M.csv', header=True, inferSchema=True)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1589357389771_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sqlContext.spark.getConf().get('spark.executor.heartbeatInterval')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'SQLContext' object has no attribute 'spark'
Traceback (most recent call last):
AttributeError: 'SQLContext' object has no attribute 'spark'



In [5]:
catCols = ['C1'] + ['C{}'.format(k) for k in range(14, 22)]
catCols += ['banner_pos', 'site_category', 'device_type', 'device_conn_type',
            'site_id', 'site_domain', 'app_id', 'app_domain', 'app_category', 'device_model']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql.functions import udf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
posMapper = udf(lambda x: 0 if x < 0 else x)
df = df.withColumn('C20_1', posMapper(df['C20']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
catCols.remove('C20')
catCols.append('C20_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.ml.feature import OneHotEncoderEstimator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from pyspark.ml.feature import StringIndexer
for c in ['site_category', 'site_id', 'site_domain', 'app_id', 'app_domain', 'app_category', 'device_model']:
    si = StringIndexer(inputCol=c, outputCol='{}_ix'.format(c))
    df = si.fit(df).transform(df)
    catCols.remove(c)
    catCols.append('{}_ix'.format(c))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.sql.types import IntegerType
df = df.withColumn("C20_1int", df['C20_1'].cast(IntegerType()))
catCols.remove('C20_1')
catCols.append('C20_1int')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
encoder = OneHotEncoderEstimator(inputCols=catCols, outputCols=[c + 'Enc' for c in catCols])
enc_model = encoder.fit(df)
encoded = enc_model.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
trainCols = [c for c in encoded.columns if c.endswith('Enc')] + ['day', 'hour', 'dayofweek']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
encoded = encoded.withColumn('label', encoded['click'].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.ml.classification import LogisticRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
va = VectorAssembler(inputCols=trainCols, outputCol='features')
encoded = va.transform(encoded)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
auc = []
recall = []
L1 = [0.00001, 0.00002, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
for l in L1:
    lr = LogisticRegression(featuresCol='features', labelCol='label', elasticNetParam=1, regParam=l)
    model = lr.fit(encoded)
    result = model.evaluate(encoded)
    auc.append(result.areaUnderROC)
    r = result.recallByLabel
    recall.append(r)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '404' from https://172.31.85.79:18888/sessions/3/statements/17 with error payload: {"msg":"Session '3' not found."}


In [20]:
auc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.7510520522355986]